In [ ]:
import warnings

import ipywidgets as widgets
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display, Latex
from scipy import optimize
from scipy import stats
from sklearn import metrics
from sklearn.linear_model import HuberRegressor, LinearRegression, RANSACRegressor

import rp2.data
from rp2 import hagai_2018, create_gene_symbol_map
from rp2.processing import calculate_df_mahalanobis_distance

rp2.check_environment()

### Settings controlling downstream analysis

In [ ]:
analysis_species = "mouse"
analysis_counts = "median"
analysis_treatments = ["unst", "lps", "pic"]
analysis_time_points = ["0", "2", "4", "6"]

mv_outlier_factor = 1.35
min_mv_r2 = 0.75
min_conditions = 12

bp_curve_loss = "huber"
bp_curve_f_scale = 1.00

remove_mv_outliers_downstream = False

index_columns = ["replicate", "treatment", "time_point"]
all_index_columns = ["gene"] + index_columns

In [ ]:
gene_symbol_map = create_gene_symbol_map(analysis_species)

### Determine which genes have a sufficient number of conditions with valid burst parameters

In [ ]:
condition_info_df = rp2.data.load_txburst_results(analysis_species, index_columns, count_type=analysis_counts)
condition_info_df["k_burstiness"] = condition_info_df.k_off / condition_info_df.k_on

condition_info_df = condition_info_df.loc[condition_info_df.treatment.isin(analysis_treatments)]
condition_info_df = condition_info_df.loc[condition_info_df.time_point.isin(analysis_time_points)]
print(f"{len(condition_info_df):,} conditions for {condition_info_df.gene.nunique():,} genes have been processed by txburst")

condition_info_df["valid_bp"] = condition_info_df.bs_point.notna() & condition_info_df.bf_point.notna()

valid_counts = condition_info_df.groupby("gene").valid_bp.agg(np.count_nonzero)
valid_gene_ids = valid_counts.index[valid_counts >= min_conditions]
print(f"{len(valid_gene_ids):,} genes have {min_conditions} or more conditions with valid burst parameters")

condition_info_df = condition_info_df.loc[condition_info_df.gene.isin(valid_gene_ids)]

### Calculate statistics of RNA counts

In [ ]:
def calculate_count_stats(condition_subset):
    counts_adata = hagai_2018.load_counts(analysis_species, scaling=analysis_counts)
    print(f"Counts available for {counts_adata.n_obs:,} cells and {counts_adata.n_vars:,} genes")

    counts_adata = counts_adata[:, counts_adata.var_names.isin(condition_subset.gene)]
    for column in index_columns:
        counts_adata = counts_adata[counts_adata.obs[column].isin(condition_subset[column])]

    counts_adata = counts_adata.copy()
    print(f"Calculating count statistics for {counts_adata.n_obs:,} cells and {counts_adata.n_vars:,} genes")

    stats_df = hagai_2018.calculate_counts_condition_stats(counts_adata, group_columns=index_columns)
    distance_df = stats_df.groupby("gene", as_index=False).apply(calculate_df_mahalanobis_distance, ["mean", "variance"], name="mv_m_dist")

    return stats_df.join(distance_df)


condition_info_df = condition_info_df.set_index(all_index_columns).join(
    calculate_count_stats(condition_info_df).set_index(all_index_columns),
    how="left",
).reset_index()

### Display an interactive mean-variance plot for genes with sufficient conditions with valid burst parameters

Although the list of genes is restricted to those with a minimum number of conditions with valid burst parameters, all conditions are plotted and used to fit the regression line. The solid line shows the fit to all points and dotted lines show fits for individual treatments (including unstimulated points in each case).

Points determined to be outliers by the regressor are indicated and the sensitivity can be adjusted using the outlier_factor slider. Changes to this value are for illustration only and will not change downstream analysis (for this, change the value of mv_outlier_factor above and re-run all cells). Outliers may be removed from downstream analysis based on the value of remove_mv_outliers_downstream.

In [ ]:
def apply_inlier_ols_regressor(df, x_var, y_var, distance_threshold, output_outliers=False):
    distance_cutoff = np.sqrt(stats.chi2.ppf(distance_threshold, 2))
    distances = df["mv_m_dist"]
    inliers_mask = (distances <= distance_cutoff).values
    if np.count_nonzero(inliers_mask) < 2:
        inliers_mask[np.argsort(distances)[:2]] = True

    all_x, all_y = df.loc[:, [x_var, y_var]].to_numpy().T
    all_x = all_x.reshape(-1, 1)
    x, y = all_x[inliers_mask], all_y[inliers_mask]

    regressor = LinearRegression().fit(x, y)
    results = {
        "slope": regressor.coef_.item(),
        "intercept": regressor.intercept_.item(),
        "r2": regressor.score(all_x, all_y),
    }
    if output_outliers:
        results["outliers"] = ~inliers_mask
        wo_r2 = regressor.score(x, y) if np.count_nonzero(inliers_mask) > 2 else np.nan
        results["r2_without_outliers"] = wo_r2
    return results


def apply_huber_regressor(df, x_var, y_var, epsilon=1.35, output_outliers=False):
    x, y = df.loc[:, [x_var, y_var]].to_numpy().T
    x = x.reshape(-1, 1)
    regressor = HuberRegressor(epsilon=epsilon).fit(x, y)
    results = {
        "slope": regressor.coef_.item(),
        "intercept": regressor.intercept_.item(),
        "r2": regressor.score(x, y),
    }
    if output_outliers:
        outliers_mask = regressor.outliers_
        inliers_mask = ~outliers_mask
        results["outliers"] = outliers_mask
        wo_r2 = regressor.score(x[inliers_mask], y[inliers_mask]) if np.count_nonzero(inliers_mask) > 2 else np.nan
        results["r2_without_outliers"] = wo_r2
    return results


def apply_ransac_regressor(df, x_var, y_var, mad_scale=1.0, output_outliers=False):
    mad = df[y_var].mad()
    x, y = df.loc[:, [x_var, y_var]].to_numpy().reshape(1, -1, 2).T
    regressor = RANSACRegressor(residual_threshold=mad * mad_scale)
    regressor.fit(x, y)
    estimator = regressor.estimator_
    results = {
        "slope": estimator.coef_.item(),
        "intercept": estimator.intercept_.item(),
        "r2": estimator.score(x, y),
    }
    if output_outliers:
        outliers_mask = ~regressor.inlier_mask_
        inliers_mask = ~outliers_mask
        results["outliers"] = outliers_mask
        wo_r2 = estimator.score(x[inliers_mask], y[inliers_mask]) if np.count_nonzero(inliers_mask) > 2 else np.nan
        results["r2_without_outliers"] = wo_r2
    return results


def make_gene_selector(gene_ids):
    gene_symbols = gene_symbol_map.lookup(gene_ids).sort_values()
    return widgets.Select(
        options=list(zip(gene_symbols.values, gene_symbols.index)),
        rows=3,
    )


def plot_points_with_outliers(x, y, outliers, colours=None):
    if colours is None:
        colours = ["black"] * len(x)

    plt.scatter(
        x[~outliers],
        y[~outliers],
        marker="o",
        c=colours[~outliers],
    )
    plt.scatter(
        x[outliers],
        y[outliers],
        marker="x",
        c=colours[outliers],
        label="outlier",
    )

    
def format_plus_c(c):
    return f"+{c:.2f}" if c > 0 else f"-{abs(c):.2f}"


apply_mv_regressor = apply_huber_regressor
apply_standard_regressor = apply_huber_regressor


@widgets.interact(gene_id=make_gene_selector(valid_gene_ids), plot_treatment_lines=False, outlier_factor=widgets.FloatSlider(mv_outlier_factor, min=1.01, max=5, step=0.01))
def plot_mean_var(gene_id, plot_treatment_lines, outlier_factor):
    treatment_colour_map = {"unst": "black", "lps": "red", "pic": "green"}

    condition_info_subset = condition_info_df.loc[condition_info_df.gene == gene_id]

    lr_results = apply_mv_regressor(condition_info_subset, "mean", "variance", outlier_factor, output_outliers=True)
    outliers = lr_results["outliers"]

    treatment_lr_results_map = {treatment: apply_huber_regressor(condition_info_subset.loc[condition_info_subset.treatment.isin(["unst", treatment])], "mean", "variance")
                                for treatment in filter(lambda t: t != "unst", analysis_treatments)}

    plot_output = widgets.Output()
    info_output = widgets.Output()

    with plot_output:
        colours = condition_info_subset.treatment.map(treatment_colour_map)
        legend_handles = [
            matplotlib.lines.Line2D([], [], marker="x", color="black", label="Outlier", linestyle="None", markersize=8)
        ]
        for treatment_name, colour in treatment_colour_map.items():
            legend_handles.append(matplotlib.lines.Line2D([], [], marker="o", color=colour, label=treatment_name, linestyle="None", markersize=8))

        x, y = condition_info_subset.loc[:, ["mean", "variance"]].to_numpy().T
        plot_points_with_outliers(x, y, outliers, colours)

        lr_x = np.asarray((0, x.max()))
        lr_y = (lr_x * lr_results["slope"]) + lr_results["intercept"]
        plt.plot(lr_x, lr_y, "-")

        if plot_treatment_lines:
            for treatment, lr_res in treatment_lr_results_map.items():
                lr_y2 = (lr_x * lr_res["slope"]) + lr_res["intercept"]
                plt.plot(lr_x, lr_y2, ":", c=treatment_colour_map[treatment])

        plt.xlim(left=0)
        plt.ylim(bottom=0)
        #plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
        plt.legend(
            handles=legend_handles,
            loc="upper left",
            bbox_to_anchor=(1, 1)
        )
        plt.show()

    with info_output:
        print(f"No. of conditions with burst parameters: {np.count_nonzero(condition_info_subset.valid_bp)} / {len(condition_info_subset)}")
        print(f"No. of outliers: {np.count_nonzero(outliers)}")
        display(Latex(f"$R^2$ with outliers: {lr_results['r2']:.3f}"))
        display(Latex(f"$R^2$ without outliers: {lr_results['r2_without_outliers']:.3f}"))
        display(Latex(f"$\sigma^2={lr_results['slope']:.2f}\mu{format_plus_c(lr_results['intercept'])}$"))

    display(widgets.HBox((plot_output, info_output)))

### Fit trends to mean-variance relationships

A linear mean-variance trend is fit and genes are further filtered by the $R^2$ score and, depending upon remove_mv_outliers_downstream, the number of conditions remaining once outliers are excluded.

In [ ]:
condition_info_df["mv_outlier"] = False
mv_gene_info_df = pd.DataFrame()

for gene_id, gene_df in condition_info_df.groupby("gene"):
    results = apply_mv_regressor(gene_df, "mean", "variance", mv_outlier_factor, output_outliers=True)

    outliers = results.pop("outliers")
    results["n_valid_bp"] = np.count_nonzero(gene_df.valid_bp)
    results["n_outliers"] = np.count_nonzero(outliers)
    if remove_mv_outliers_downstream:
        results["r2"] = results.pop("r2_without_outliers")

    mv_gene_info_df = mv_gene_info_df.append(pd.DataFrame(index=[gene_id], data=results))
    condition_info_df.loc[gene_df.index, "mv_outlier"] = outliers

mv_gene_info_df.index.name = "gene"

print(f"Of {len(condition_info_df):,} conditions:")
print(f"  {mv_gene_info_df.n_outliers.sum():,} are mean-variance outliers")

print(f"Of {len(mv_gene_info_df):,} genes:")

subset_with_good_r2 = mv_gene_info_df.loc[mv_gene_info_df.r2 >= min_mv_r2]
print(f"  {len(subset_with_good_r2):,} have R2 >= {min_mv_r2}")
filtered_gene_ids = subset_with_good_r2.index

if remove_mv_outliers_downstream:
    subset_with_good_n_conditions = mv_gene_info_df.loc[(mv_gene_info_df.n_valid_bp - mv_gene_info_df.n_outliers) >= min_conditions]
    print(f"  {len(subset_with_good_n_conditions):,} have >= {min_conditions} non-outliers with valid burst point estimates")
    filtered_gene_ids = filtered_gene_ids[filtered_gene_ids.isin(subset_with_good_n_conditions.index)]

print(f"  {len(filtered_gene_ids):,} will be kept")
print("The following genes will be filtered out:")
display(gene_symbol_map.added_to(mv_gene_info_df.loc[~mv_gene_info_df.index.isin(filtered_gene_ids)]).sort_values(by="gene_symbol"))

### Display an interactive mean-burst parameter plot for filtered genes

In [ ]:
def get_bp_condition_subset(condition_df):
    condition_mask = condition_df.valid_bp
    if remove_mv_outliers_downstream:
        condition_mask &= ~condition_df.mv_outlier
    return condition_df.loc[condition_mask]


def power_function(x, a, b, c):
    return (a * np.power(x, b)) + c


def power_function_loss(params, x, y):
    a, b, c = params
    return power_function(x, a, b, c) - y


def calculate_curve_fit(df, x_var, y_var, loss_function, f_scale):
    x, y = df.loc[:, [x_var, y_var]].to_numpy().T

    ls_results = optimize.least_squares(
        power_function_loss,
        [1, 1, 0],
        args=(x, y),
        loss=loss_function,
        f_scale=f_scale,
    )
    if ls_results.success:
        a, b, c = ls_results.x
        r2 = metrics.r2_score(y, power_function(x, a, b, c))
    else:
        a = b = c = r2 = np.nan

    return {
        "a": a,
        "b": b,
        "c": c,
        "r2": r2,
    }


@widgets.interact(
    gene_id=make_gene_selector(filtered_gene_ids),
    parameter=widgets.Select(options=[("Frequency", "f"), ("Size", "s")], rows=2),
    loss_function=widgets.Select(options=["linear", "cauchy", "huber", "soft_l1"], value=bp_curve_loss, rows=4),
    f_scale=widgets.FloatSlider(bp_curve_f_scale, min=0.01, max=5, step=0.01),
)
def plot_mean_bp(gene_id, parameter, loss_function, f_scale):
    x_var = "mean"
    y_var = f"b{parameter}_point"

    condition_info_subset = get_bp_condition_subset(condition_info_df.loc[condition_info_df.gene == gene_id])

    curve_res = calculate_curve_fit(condition_info_subset, x_var, y_var, loss_function, f_scale)
    line_res = apply_standard_regressor(condition_info_subset, x_var, y_var)

    x_range = condition_info_subset[x_var].min(), condition_info_subset[x_var].max()

    plot_output = widgets.Output()
    info_output = widgets.Output()

    with plot_output:
        ax = sns.scatterplot(
            x=x_var,
            y=y_var,
            hue="treatment",
            data=condition_info_subset,
        )

        line_x = np.asarray(x_range)
        line_y = (line_x * line_res["slope"]) + line_res["intercept"]
        ax.plot(line_x, line_y, ls=":")

        if curve_res["a"] is not np.nan:
            curve_x = np.linspace(*x_range)
            curve_y = power_function(curve_x, curve_res["a"], curve_res["b"], curve_res["c"])
            ax.plot(curve_x, curve_y, ls="-")

        ax.set_xlim(0, x_range[1])
        ax.set_ylim(0, condition_info_subset[y_var].max())

        plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
        plt.show()

    with info_output:
        display(Latex(f"Linear $R^2$: {line_res['r2']:.3f}"))
        display(Latex(f"Curve $R^2$: {curve_res['r2']:.3f}"))

    display(widgets.HBox((plot_output, info_output)))

### Fit trends to burst parameters

Lines and curves are fit for the filtered list of genes based on conditions with valid point estimates of burst parameters. Mean-variance outliers may be excluded from the fitting based on the value of remove_mv_outliers_downstream.

In [ ]:
def calculate_per_gene_info(df):
    lr_dict = {
        "bf": apply_standard_regressor(df, "mean", "bf_point"),
        "bs": apply_standard_regressor(df, "mean", "bs_point"),
    }

    curve_dict = {
        "bf": calculate_curve_fit(df, "mean", "bf_point", bp_curve_loss, bp_curve_f_scale),
        "bs": calculate_curve_fit(df, "mean", "bs_point", bp_curve_loss, bp_curve_f_scale),
    }

    results_dict = {
        "n_conditions": len(df),
    }
    for lr_n, lr_v in lr_dict.items():
        for n, v in lr_v.items():
            results_dict[f"{lr_n}_{n}"] = v
    for curve_n, curve_v in curve_dict.items():
        for n, v in curve_v.items():
            results_dict[f"{curve_n}_pf_{n}"] = v

    return pd.Series(results_dict)


condition_info_gene_subset = condition_info_df.loc[condition_info_df.gene.isin(filtered_gene_ids)]
condition_mask = condition_info_gene_subset.valid_bp
if remove_mv_outliers_downstream:
    condition_mask &= ~condition_info_gene_subset.mv_outlier
condition_info_gene_subset = condition_info_gene_subset.loc[condition_mask].groupby("gene").apply(calculate_per_gene_info).join(
    mv_gene_info_df.rename(columns={n: f"mv_{n}" for n in mv_gene_info_df.columns}),
)

print("Curves successfully fit to:")
print(f"  Burst sizes of {condition_info_gene_subset.bs_pf_a.count():,} genes")
print(f"  Burst frequencies of {condition_info_gene_subset.bf_pf_a.count():,} genes")

### Display a scatter plot of $R^2$ values for burst size against frequency to indicate dominant modulation

In [ ]:
ax = sns.scatterplot(
    x="bs_r2",
    y="bf_r2",
    data=condition_info_gene_subset,
)
plt.plot((-0.5, 1), (-0.5, 1), "-")
plt.axvline(x=0, ls=":")
plt.axhline(y=0, ls=":")
ax.set_aspect(1)
plt.show()

print(f"{np.count_nonzero(condition_info_gene_subset.bf_r2 > condition_info_gene_subset.bs_r2):,} points above line")
print(f"{np.count_nonzero(condition_info_gene_subset.bf_r2 < condition_info_gene_subset.bs_r2):,} points below line")

### Show plots of trends against mean RNA for filtered genes

Genes are sorted by descending $R^2$ value of the burst frequency linear regression fit. Highlighted outliers are outliers of the mean-variance fit.

In [ ]:
def plot_relationship_scatter(ax, condition_info, y_var):
    x = condition_info.loc[:, "mean"]
    y = condition_info.loc[:, y_var]

    ax.scatter(
        x[~condition_info.mv_outlier],
        y[~condition_info.mv_outlier],
        marker="o",
    )
    ax.scatter(
        x[condition_info.mv_outlier],
        y[condition_info.mv_outlier],
        marker="x",
        label="mv outlier",
    )
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=0)
    ax.set_xlabel("mean")
    ax.set_ylabel(y_var)


def plot_relationship_line(ax, condition_info, lr_info, y_var_prefix):
    x_range = np.asarray((0, condition_info["mean"].max()))
    ax.plot(x_range, (x_range * lr_info[f"{y_var_prefix}_slope"]) + lr_row[f"{y_var_prefix}_intercept"])

    r2 = lr_info[f"{y_var_prefix}_r2"]
    ax.set_title(f"$R^2=${r2:.2f}")


def plot_relationship_curve(ax, condition_info, lr_info, y_var_prefix):
    a, b, c = [lr_info[f"{y_var_prefix}_pf_{coef}"] for coef in ["a", "b", "c"]]
    if a is np.nan: return

    x = np.linspace(np.finfo(np.float).eps, condition_info["mean"].max())
    y = power_function(x, a, b, c)

    ax.plot(x, y, "--")


lr_to_plot = condition_info_gene_subset.sort_values(by="bf_r2", ascending=False)
for idx, (gene_id, lr_row) in enumerate(lr_to_plot.iterrows(), start=1):
    print(f"{idx}. {gene_symbol_map.lookup(gene_id)}")
    display(lr_row.to_frame().T)

    condition_info_subset = condition_info_df.loc[condition_info_df.gene == gene_id]

    _, axes = plt.subplots(ncols=4, figsize=(16, 4), sharex=True)

    plot_relationship_scatter(axes[0], condition_info_subset, "variance")
    plot_relationship_line(axes[0], condition_info_subset, lr_row, "mv")

    valid_bp_condition_info = condition_info_subset.loc[condition_info_subset.valid_bp]

    for prefix, ax in zip(("bf", "bs"), axes[1:3]):
        plot_relationship_scatter(ax, valid_bp_condition_info, f"{prefix}_point")
        plot_relationship_line(ax, valid_bp_condition_info, lr_row, prefix)
        plot_relationship_curve(ax, valid_bp_condition_info, lr_row, prefix)

    plot_relationship_scatter(axes[3], valid_bp_condition_info, "k_burstiness")
    axes[3].set_ylim(bottom=valid_bp_condition_info.k_burstiness.min())
    axes[3].set_yscale("log")

    plt.tight_layout()
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()